# Tarea 3

In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
ratings = pd.read_csv('../data/ratings.csv')
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies = pd.read_csv('../data/movies.csv')
print(movies.shape)
movies.head()

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## ¿Cuáles fueron los géneros mejor rankeados en promedio en el año 2005?

In [4]:
movies_groupby = ratings.groupby(['movieId'])

rated_movies = movies_groupby.mean().merge(movies, on='movieId').drop(columns=['userId', 'timestamp'])
# rated_movies.sort_values('rating', ascending=False).head()

In [5]:
rated_movies['genres'] = rated_movies['genres'].str.split('|')
# rated_movies.head()

In [6]:
# from https://stackoverflow.com/questions/40569402/pandas-create-several-rows-from-column-that-is-a-list
def melt_series(s):
    lengths = s.str.len().values
    flat = [i for i in itertools.chain.from_iterable(s.values.tolist())]
    idx = np.repeat(s.index.values, lengths)
    return pd.Series(flat, idx, name=s.name)

splited_genres = rated_movies.drop(columns=['genres']).join(melt_series(rated_movies.genres).to_frame()).reset_index(drop=True)
movies_2005 = splited_genres[splited_genres.title.str.endswith('(2005)')]
movies_2005.head()

,movieId,rating,title,genres
12385,27618,2.75,"Sound of Thunder, A (2005)",Action
12386,27618,2.75,"Sound of Thunder, A (2005)",Adventure
12387,27618,2.75,"Sound of Thunder, A (2005)",Drama
12388,27618,2.75,"Sound of Thunder, A (2005)",Sci-Fi
12389,27618,2.75,"Sound of Thunder, A (2005)",Thriller


In [7]:
best_genres = movies_2005.drop(columns=['movieId', 'title']).groupby('genres').mean().sort_values(by='rating', ascending=False)
best_genres.head()

,rating
genres,
Film-Noir,4.077381
War,3.781869
Documentary,3.773495
Animation,3.443075
IMAX,3.426322


## ¿Cuál es el top 5 de películas mejor rankeadas en promedio?

In [8]:
rated_movies.sort_values(by='rating', ascending=False).head()

,movieId,rating,title,genres
7638,88448,5.0,Paper Birds (Pájaros de papel) (2010),"[Comedy, Drama]"
8089,100556,5.0,"Act of Killing, The (2012)",[Documentary]
9065,143031,5.0,Jump In! (2007),"[Comedy, Drama, Romance]"
9076,143511,5.0,Human (2015),[Documentary]
9078,143559,5.0,L.A. Slasher (2015),"[Comedy, Crime, Fantasy]"


In [9]:
# Películas con promedio de rating == 5.0
len(rated_movies.loc[rated_movies.rating == 5])

296

## ¿Cuál es el género con más reviews?

In [44]:
ratings_count = splited_genres.drop(columns=['rating', 'title']).merge(ratings, on='movieId')
ratings_count = ratings_count.drop(columns=['movieId', 'userId','timestamp']).groupby('genres').count()
ratings_count.sort_values(by='rating', ascending=False).head(3)

,rating
genres,
Drama,41928
Comedy,39053
Action,30635


## ¿Cuáles es el género por año con mejor rating entre el año 2000 y 2010?

In [11]:
years = range(2000, 2011)
genre_by_year = {}

for y in years:
    movies_ = splited_genres[splited_genres.title.str.endswith(f'({y})')]
    best_genre = movies_.drop(columns=['movieId', 'title']).groupby('genres').mean().sort_values(by='rating', ascending=False).head(1).index[0]
    genre_by_year[y] = best_genre

genre_by_year

{2000: 'War',
 2001: 'Documentary',
 2002: 'Documentary',
 2003: 'Documentary',
 2004: 'Documentary',
 2005: 'Film-Noir',
 2006: 'Documentary',
 2007: 'Western',
 2008: 'War',
 2009: 'Western',
 2010: 'Mystery'}

## ¿Cuál es el usuario con más críticas y cuál es su género favorito?

In [12]:
ratings_per_user = ratings.groupby(['userId']).count()
user = ratings_per_user['rating'].sort_values(ascending=False).head().index[0]
user = ratings.loc[ratings.userId == user].drop(columns=['userId', 'timestamp'])
user = user.merge(splited_genres.drop(columns=['rating', 'title']), on='movieId').drop(columns=['movieId']).groupby('genres').mean()
user.sort_values(by='rating', ascending=False).head(3)

,rating
genres,
(no genres listed),4.250000
Film-Noir,3.857143
War,3.830645


## ¿Cuál es el género con más películas y cuál es su promedio de reviews?

In [90]:
# movies_by_genre = splited_genres.drop(columns=['movieId', 'title'])
movies_by_genre = movies.drop(columns=['genres', 'title']).join(melt_series(movies.genres.str.split('|'))).groupby('genres').count()
genres_rating = splited_genres.drop(columns=['movieId', 'title']).groupby('genres').mean()

movies_by_genre = movies_by_genre.merge(genres_rating, on='genres', suffixes=('_count', '_mean')).rename(columns={'movieId':'movie_count'})

movies_by_genre.sort_values(by='movie_count', ascending=False).head(3)

,movie_count,rating
genres,,
Drama,4361,3.420905
Comedy,3756,3.181716
Thriller,1894,3.156116


## ¿Qué películas no tienen ningún review?

In [113]:
ratings_by_movie = ratings.drop(columns=['rating', 'timestamp']).groupby('movieId').count()
movies_ratings = movies.drop(columns=['genres']).merge(ratings_by_movie, on='movieId', how='left')

movies_ratings[movies_ratings['userId'].isna()]

,movieId,title,userId
816,1076,"Innocents, The (1961)",NaN
2211,2939,Niagara (1953),NaN
2499,3338,For All Mankind (1989),NaN
2587,3456,"Color of Paradise, The (Rang-e khoda) (1999)",NaN
3118,4194,I Know Where I'm Going! (1945),NaN
4037,5721,"Chosen, The (1981)",NaN
4506,6668,"Road Home, The (Wo de fu qin mu qin) (1999)",NaN
4598,6849,Scrooge (1970),NaN
4704,7020,Proof (1991),NaN
5020,7792,"Parallax View, The (1974)",NaN
